# Sintetizacion de Voz usando valores discretos


### Encodec

El modelo de códec neuronal EnCodec fue propuesto en Compresión de audio neuronal de alta fidelidad por Alexandre Défossez, Jade Copet, Gabriel Synnaeve y Yossi Adi.

Aquí hay un ejemplo rápido de cómo codificar y decodificar un audio usando este modelo:

In [ ]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor

librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
librispeech_dummy = librispeech_dummy.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
audio_sample = librispeech_dummy[-1]["audio"]["array"]
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]
# or the equivalent with a forward pass
audio_values = model(inputs["input_values"], inputs["padding_mask"]).audio_values

Ahora, analizemos el interior de Encodec

### Bark

Ahora, utilizaremos bark (de suno-ai https://github.com/suno-ai/bark) para generar sonidos.
Y luego analizamos el contenido interno.
A continuación se muestra una lista de algunos sonidos conocidos ajenos al habla, pero cada día encontramos más.
- [laughter]
- [laughs]
- [sighs]
- [music]
- [gasps]
- [clears throat]
- — or ... por vacilaciones
- ♪ para letras de canciones
- MAYÚSCULAS para enfatizar una palabra
- [MAN] y [WOMAN] direccionar a Bark hacia hablantes masculinos y femeninos, respectivamente

Puedes obtener la lista de los preset aqui: https://suno-ai.notion.site/8b8e8749ed514b0cbf3f699013548683?v=bc67cff786b04b50b3ceb756fd05f68c

In [ ]:
import torch
import soundfile as sf
from datetime import datetime
from transformers import AutoProcessor, BarkModel


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16

processor = AutoProcessor.from_pretrained(
    "suno/bark-small",
    cache_dir=CACHE_DIR,
)
model = BarkModel.from_pretrained(
    "suno/bark",
    cache_dir="./models",
).to(device)

model.enable_cpu_offload()
sampling_rate = model.generation_config.sample_rate

caption = ""
voice_preset = "v2/es_speaker_1"

inputs = processor(caption, voice_preset=voice_preset).to(device)
audio_out = model.generate(**inputs).squeeze().cpu().numpy()

time_now = datetime.now().strftime("%Y%m%d_%H%M%S")
name_out = f"generated/speech_{time_now}.wav"
sf.write(name_out, audio_out, sampling_rate)